In [1]:
"""
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session
"""

'\n# This Python 3 environment comes with many helpful analytics libraries installed\n# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python\n# For example, here\'s several helpful packages to load\n\nimport numpy as np # linear algebra\nimport pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)\n\n# Input data files are available in the read-only "../input/" directory\n# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory\n\nimport os\nfor dirname, _, filenames in os.walk(\'/kaggle/input\'):\n    for filename in filenames:\n        print(os.path.join(dirname, filename))\n\n# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" \n# You can also write temporary files to /kaggle/temp/, but they won\'t be saved outside of the current session\n'

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

path="others/"
initial_lgb_submission = pd.read_csv(path+'initial_lgb_submission.csv')
initial_lgb_submission

,ID,item_cnt_month
0,0,0.536405
1,1,2.151394
2,2,0.913563
3,3,0.721768
4,4,5.372386
...,...,...
214195,214195,0.125985
214196,214196,0.032229
214197,214197,0.074373
214198,214198,0.018177


In [2]:
blended_submission1 = pd.read_csv(path+'blended_submission1.csv')
blended_submission1

,ID,item_cnt_month
0,0,0.710527
1,1,0.346980
2,2,1.127346
3,3,0.457653
4,4,3.548673
...,...,...
214195,214195,0.104466
214196,214196,0.015815
214197,214197,0.050054
214198,214198,0.009008


# 데이터 로드 및 훈련, 테스트 데이터 확인

In [4]:
# Basic data loading
path = "data/"

df_train = pd.read_csv(path + 'sales_train.csv')
df_test = pd.read_csv(path + 'test.csv', index_col='ID')
df_shops = pd.read_csv(path + 'shops.csv', index_col='shop_id')
df_items = pd.read_csv(path + 'items.csv', index_col='item_id')
df_itemcat = pd.read_csv(path + 'item_categories.csv', index_col='item_category_id')
sample_submission = pd.read_csv(path + 'sample_submission.csv', index_col='ID')

In [5]:
# 시험용으로 변수 수 줄이기
df_train_drop = df_train[:int(df_train.shape[0]/10)]

In [6]:
df_train_drop

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,02.01.2013,0,59,22154,999.00,1.0
1,03.01.2013,0,25,2552,899.00,1.0
2,05.01.2013,0,25,2552,899.00,-1.0
3,06.01.2013,0,25,2554,1709.05,1.0
4,15.01.2013,0,25,2555,1099.00,1.0
...,...,...,...,...,...,...
293579,09.03.2013,2,8,3960,299.00,1.0
293580,24.03.2013,2,8,3955,1649.00,1.0
293581,24.03.2013,2,8,3921,299.00,1.0
293582,21.03.2013,2,8,3823,399.00,1.0


In [7]:
from sklearn.preprocessing import LabelEncoder
from datetime import datetime

def date_processing(df):
    df["date"] = df["date"].apply(lambda date_str : datetime.strptime(date_str, "%d.%m.%Y"))

    df["year"] = df["date"].apply(lambda date : date.year)
    df["month"] = df["date"].apply(lambda date : date.month)
    df["day"] = df["date"].apply(lambda date : date.day)
    df = df.drop(columns=["date"])
    df = df[["year", "month","day", "date_block_num", "shop_id", "item_id",
             "item_price", "item_cnt_day"]]
    return df


df_train_preprocessed = df_train_drop
df_train_preprocessed = date_processing(df_train_preprocessed)
df_train_preprocessed.head()

/home/aiffel/anaconda3/envs/aiffel/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/home/aiffel/anaconda3/envs/aiffel/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/home/aiffel/anaconda3/envs/aiffel/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value i

,year,month,day,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,2013,1,2,0,59,22154,999.00,1.0
1,2013,1,3,0,25,2552,899.00,1.0
2,2013,1,5,0,25,2552,899.00,-1.0
3,2013,1,6,0,25,2554,1709.05,1.0
4,2013,1,15,0,25,2555,1099.00,1.0


In [8]:
df_train_preprocessed.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 293584 entries, 0 to 293583
Data columns (total 8 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   year            293584 non-null  int64  
 1   month           293584 non-null  int64  
 2   day             293584 non-null  int64  
 3   date_block_num  293584 non-null  int64  
 4   shop_id         293584 non-null  int64  
 5   item_id         293584 non-null  int64  
 6   item_price      293584 non-null  float64
 7   item_cnt_day    293584 non-null  float64
dtypes: float64(2), int64(6)
memory usage: 17.9 MB


In [9]:
df_train_preprocessed

,year,month,day,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,2013,1,2,0,59,22154,999.00,1.0
1,2013,1,3,0,25,2552,899.00,1.0
2,2013,1,5,0,25,2552,899.00,-1.0
3,2013,1,6,0,25,2554,1709.05,1.0
4,2013,1,15,0,25,2555,1099.00,1.0
...,...,...,...,...,...,...,...,...
293579,2013,3,9,2,8,3960,299.00,1.0
293580,2013,3,24,2,8,3955,1649.00,1.0
293581,2013,3,24,2,8,3921,299.00,1.0
293582,2013,3,21,2,8,3823,399.00,1.0


In [18]:
#df_train_preprocessed["item_cnt_month"] = df_train_preprocessed.groupby(["year", "month", "shop_id", "item_id"])["item_cnt_day"].sum().reset_index()["item_cnt_day"]

In [10]:
cols = ["year", "month","date_block_num", "shop_id", "item_id"]
df_train_month = df_train_preprocessed.groupby(cols)["item_cnt_day"].sum().reset_index()

In [11]:
df_train_month

,year,month,date_block_num,shop_id,item_id,item_cnt_day
0,2013,1,0,0,32,6.0
1,2013,1,0,0,33,3.0
2,2013,1,0,0,35,1.0
3,2013,1,0,0,43,1.0
4,2013,1,0,0,51,2.0
...,...,...,...,...,...,...
159901,2013,3,2,57,5948,2.0
159902,2013,3,2,57,5964,1.0
159903,2013,3,2,57,5998,1.0
159904,2013,3,2,57,6002,1.0


In [12]:
df_train_month = df_train_month.rename(columns={"item_cnt_day": "item_cnt_month"})
df_train_month

,year,month,date_block_num,shop_id,item_id,item_cnt_month
0,2013,1,0,0,32,6.0
1,2013,1,0,0,33,3.0
2,2013,1,0,0,35,1.0
3,2013,1,0,0,43,1.0
4,2013,1,0,0,51,2.0
...,...,...,...,...,...,...
159901,2013,3,2,57,5948,2.0
159902,2013,3,2,57,5964,1.0
159903,2013,3,2,57,5998,1.0
159904,2013,3,2,57,6002,1.0


In [13]:
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
random_state = 200

month_X = df_train_month.iloc[:,:5]
month_y = df_train_month.iloc[:,5]

def month_predict(model, month_X, month_y, kind="default", print_len=7):
    X_train, X_val, y_train, y_val = train_test_split(month_X, month_y, test_size=0.2, random_state=random_state)
    model.fit(X_train, y_train)
    y_pred_train = model.predict(X_train)
    model_name = model.__class__.__name__
    print("#### " +model_name+" month prediction("+kind+") ####")
    print("y_train[:{}] : {}".format(print_len, np.round(y_train[:print_len].values,1)))
    print("y_pred_train[:{}] : {}".format(print_len, np.round(y_pred_train[:print_len],1)))
    print()
    y_pred_val = model.predict(X_val)
    print("y_val[:{}] : {}".format(print_len, np.round(y_val[:print_len].values,1)))
    print("y_pred_val[:{}] : {}".format(print_len, np.round(y_pred_val[:print_len],1)))
    rmse = np.sqrt(mean_squared_error(y_pred_val, y_val))
    print("rmse : {:.6f}".format(rmse))
    print("----------\n")
    return model

In [14]:
from sklearn.ensemble import RandomForestRegressor
month_regressor = RandomForestRegressor(random_state=random_state)
month_regressor = month_predict(month_regressor, month_X, month_y)

#### RandomForestRegressor month prediction(default) ####
y_train[:7] : [1. 1. 1. 3. 1. 2. 1.]
y_pred_train[:7] : [1.4 1.1 1.  3.9 1.9 1.7 1.1]

y_val[:7] : [4. 7. 1. 8. 2. 8. 3.]
y_pred_val[:7] : [2.7 4.7 2.1 3.6 1.1 5.6 1.7]
rmse : 2.942642
----------



In [15]:
df_test["year"] = 2015
df_test["month"] = 11
df_test["date_block_num"] = 34

df_test = df_test[["year", "month", "date_block_num", "shop_id", "item_id"]]
df_test

,year,month,date_block_num,shop_id,item_id
ID,,,,,
0,2015,11,34,5,5037
1,2015,11,34,5,5320
2,2015,11,34,5,5233
3,2015,11,34,5,5232
4,2015,11,34,5,5268
...,...,...,...,...,...
214195,2015,11,34,45,18454
214196,2015,11,34,45,16188
214197,2015,11,34,45,15757


In [16]:
df_test["item_cnt_month"] = month_regressor.predict(df_test)
df_test

,year,month,date_block_num,shop_id,item_id,item_cnt_month
ID,,,,,,
0,2015,11,34,5,5037,1.93
1,2015,11,34,5,5320,1.67
2,2015,11,34,5,5233,1.30
3,2015,11,34,5,5232,1.30
4,2015,11,34,5,5268,1.94
...,...,...,...,...,...,...
214195,2015,11,34,45,18454,1.11
214196,2015,11,34,45,16188,2.62
214197,2015,11,34,45,15757,1.01


In [17]:
sample_submission["item_cnt_month"] = df_test["item_cnt_month"]
sample_submission

,item_cnt_month
ID,
0,1.93
1,1.67
2,1.30
3,1.30
4,1.94
...,...
214195,1.11
214196,2.62
214197,1.01


In [18]:
name = "0209-2210-simple-model-small-rmse-2_942"
sample_submission.to_csv("submission-"+name+".csv")